In [6]:
import numpy as np
from collections import defaultdict
#from numpy import random
#import random
import networkx as nx
import matplotlib.pyplot as plt
from IPython.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

import gurobipy as gp
from gurobipy import GRB
from gurobipy import*
from gurobipy import quicksum

import pickle

import math

In [7]:
class UCBM_model:
    
    def __init__(self, random_choice = 0, T = 3, J = 5, L = 5, B = 10, Wb = 5, delta_t = 1, Db = 5, **kwargs):
        
        # Set of time periods, buses, and transmission lines
        self.T = T
        # time interval length
        self.delta_t = delta_t # hrs

        self.el = np.array([0.37700,
                            0.36841,
                            0.36360,
                            0.36312,
                            0.36729,
                            0.37668,
                            0.38057,
                            0.38675,
                            0.39888,
                            0.40566,
                            0.40973,
                            0.41175,
                            0.41016,
                            0.40849,
                            0.40862,
                            0.40809,
                            0.41453,
                            0.44613,
                            0.48752,
                            0.48220,
                            0.46604,
                            0.43900,
                            0.40882,
                            0.38380])
                            
        
        
        self.modified_el = self.el/np.max(self.el)
        
        #1	    2	    3	4	    5	6	    7	    8	    9	    10	    11	    12	    13	    14			ccost($/mile)	life(yrs)	
        #fbus	tbus	r	x	    b	rateA	rateB	rateC	ratio	angle	status	angmin	angmax	cons			miles		2.50E+06	40	   
        """
        self.Tx_info = np.array([																					
        [1, 	2,	    0,	0.4,	0,	100,	100,	100,	0,	    0,	    1,  	-360,	360,    40],				
        [1, 	4,	    0,	0.6,	0,	80,	    80,	    80, 	0,	    0,	    1,  	-360,	360,    60],			
        [1, 	5,  	0,	0.2,	0,	100,	100,	100,	0,	    0,  	1,  	-360,	360,	20],				
        [2, 	3,  	0,	0.2,	0,	100,	100,	100,	0,  	0,  	1,  	-360,	360,	20],				
        [2, 	4,  	0,	0.4,    0,	100,	100,	100,	0,  	0,  	1,  	-360,	360,	40],				
        [3, 	5,  	0,	0.2,	0,	100,	100,	100,	0,  	0,  	1,  	-360,	360,	20],				
        [3, 	5,  	0,	0.2,	0,	100,	100,	100,	0,  	0,  	1,  	-360,	360,	20],				
        [2, 	6,  	0,	0.3,	0,	100,	100,	100,	0,  	0,  	1,  	-360,	360,	30],			
        [2, 	6,  	0,	0.3,	0,	100,	100,	100,	0,  	0,  	1,  	-360,	360,	30],				
        [2, 	6,  	0,	0.3,	0,	100,	100,	100,	0,  	0,  	1,  	-360,	360,	30],				
        [2, 	6,  	0,	0.3,	0,	100,	100,	100,	0,  	0,  	1,  	-360,	360,	30],
        [4, 	6,  	0,	0.3,	0,	100,	100,	100,	0,  	0,  	1,  	-360,	360,	30],				
        [4, 	6,  	0,	0.3,	0,	100,	100,	100,	0,  	0,  	1,  	-360,	360,	30]])
        
        """
        #modified info
        self.Tx_info = np.array([																					
        [1, 	2,	    0,	0.4,	0,	100,	100,	100,	0,	    0,	    1,  	-360,	360,    40],				
        [1, 	4,	    0,	0.6,	0,	80,	    80,	    80, 	0,	    0,	    1,  	-360,	360,    60],			
        [1, 	5,  	0,	0.2,	0,	100,	100,	100,	0,	    0,  	1,  	-360,	360,	20],				
        [2, 	3,  	0,	0.2,	0,	100,	100,	100,	0,  	0,  	1,  	-360,	360,	20],				
        [2, 	4,  	0,	0.4,    0,	100,	100,	100,	0,  	0,  	1,  	-360,	360,	40],				
        [3, 	5,  	0,	0.1,	0,	200,	100,	100,	0,  	0,  	1,  	-360,	360,	20],								
        [2, 	6,  	0,	0.075,	0,	400,	100,	100,	0,  	0,  	1,  	-360,	360,	30],			
        [4, 	6,  	0,	0.15,	0,	200,	100,	100,	0,  	0,  	1,  	-360,	360,	30]])
        
        
        # amount of bus pair in transmission line
        self.Tx = self.Tx_info.shape[0]
        
        # From_bus and To_bus list $mathcal{L}$
        self.Tx_list = [(int(self.Tx_info[0,0]), int(self.Tx_info[0,1]))]
        
        # Impedence list between every buses $x$
        self.Tx_impd_list = [self.Tx_info[0,3]]
                
        # Transmission capacity between every buses  $overline{F}$
        self.Tx_cap_list = [self.Tx_info[0,5] / 100 ]
        
        self.nx_edge_label = {(self.Tx_info[0,0],self.Tx_info[0,1]): 1}
        
        for i in range(1, self.Tx):
            
            Next_item = (int(self.Tx_info[i,0]), int(self.Tx_info[i,1]))
                 
            try:
                
                Next_item_index = self.Tx_list.index(Next_item)
                
                # Impedence paralleling add up
                self.Tx_impd_list[Next_item_index] = 1/(1/self.Tx_impd_list[Next_item_index] + 1/self.Tx_info[i,3])
                
                # capacity straight add up 
                self.Tx_cap_list[Next_item_index] += self.Tx_info[i,5] / 100

                
            except:
                
                # add new bus pairs (Tx line), imdepence and capacity info
                self.Tx_list.append( (int(self.Tx_info[i,0]), int(self.Tx_info[i,1])) )
                
                self.Tx_impd_list.append(self.Tx_info[i,3])
                
                self.Tx_cap_list.append(self.Tx_info[i,5] / 100 )
                
                self.nx_edge_label[Next_item] = len(self.Tx_impd_list)
                
        print()
        
    #	1	    2	    3	    4	5	6	7	    8	9	10	    11	    12	    13								
     #	bus_i	type	Pd	    Qd	Gs	Bs	area	Vm	Va	baseKV	zone	Vmax	Vmin				
        
        self.Bus_info =np.array([
        [1,	    3,	    80,	    0,	0,	0,	1,	    1,	0,	230,	1,	    0.95,	1.05],					
        [2,	    2,	    240,	0,	0,	0,	1,	    1,	0,	230,	1,	    0.95,	1.05],								
        [3,	    2,	    40,	    0,	0,	0,	1,	    1,	0,	230,	1,	    0.95,	1.05],								
        [4,	    1,	    160,	0,	0,	0,	1,	    1,	0,	230,	1,	    0.95,	1.05],								
        [5,	    1,	    240,	0,	0,	0,	1,	    1,	0,	230,	1,	    0.95,	1.05],								
        [6,	    1,	    0,	    0,	0,	0,	1,	    1,	0,	230,	1,	    0.95,	1.05],			
        ])
        
        
        
        # amount of Bus and the indices they possess ?
        self.Bus = self.Bus_info.shape[0]
        self.Bus_list = np.int_(self.Bus_info[:,0].flatten())
        
        # Bus_type_list
        self.Bus_type_list = self.Bus_info[:,1].flatten()
        
        # Bus_Pd_list:  Active power(Pd) at demand side(D) of each bus 
        self.Bus_Pd_list = self.Bus_info[:,2].flatten()
        
        # Bus demand at every time interval [Bus][time_interval]
        self.Bus_Pdt =  np.kron(self.Bus_Pd_list.reshape((-1,1)) , self.modified_el) / 100
        
        # Bus_Qd_list:  Reactive power(Qd) at demand side(D) of each bus 
        self.Bus_Qd_list = self.Bus_info[:,2].flatten()
        
        # Bus_area_list: N/A
        self.Bus_Area_list = self.Bus_info[:,6].flatten()
        
        # Bus_Vm_list  normalized to 1KV
        self.Bus_Vm_list = self.Bus_info[:,7].flatten()
        
        # Bus_Va_list  angle
        self.Bus_Vm_list = self.Bus_info[:,8].flatten() /360 * 2 * math.pi
        
        # Bus_BaseKV_list
        self.Bus_BV_list = self.Bus_info[:,9].flatten()
        
        # Bus_V_UB and LB  
        self.Bus_BV_UB = self.Bus_info[:,11].flatten()
        self.Bus_BV_LB = self.Bus_info[:,12].flatten()
        
        
        
        #1      2	    3	    4	    5	    6	    7	    8	9   10      11  12	        13	        14?    				
        #bus    reg	    pmax	pmin	ru	    rd	    su	    sd	tup	tdown	u0  ocost_st	a	        b(*p)	c(*p^2)	 ##x0	ccost	life	p_fo	SO2	CO2	

        self.Genthm_info = np.array([																					
	    [1,	    1,	    50,	    20,	    30,	    30,	    5*100,	0,	2,	1,	    1,	0,	        0.5*100,	20*100, 	0,	0,	    0,	    0,	    0,	0,	0],
	    [3,	    1,	    165,	85,	    60,	    60,	    10*100,	0,	2,	1,	    1,	0,	        1.5*100,	15*100,	    0,	0,	    0,	    0,	    0,	0,	0],
        [6,	    1,	    545,    280,	180,	180,	30*100,	0,	2,	1,	    1,	0,	        5.0*100,	17.5*100,	0,	0,	    0,	    0,	    0,	0,	0],
        ])
        
        # amount of Generator and the indices they possess
        self.G_I = self.Genthm_info.shape[0]
        self.G_I_list = self.Genthm_info[:,0].flatten()
        
        # Generator Power limit
        self.P_UB = self.Genthm_info[:,2].flatten() / 100
        self.P_LB = self.Genthm_info[:,3].flatten() / 100
        
        # Ramping up (operating/cold start) limitation for unit i. [5]
        self.RU_O_UB = self.P_UB - self.P_LB
        self.RU_CS_UB = self.P_UB - self.P_LB
        
        # Ramping down (operating/hot close) limitation for unit i. [6 ]
        self.RD_O_UB = self.P_UB - self.P_LB
        self.RD_HS_UB = self.P_UB - self.P_LB
        
        # startup and shut down Cost SU_{i,t} SD_{i,t} 
        self.G_ST_cost =  self.Genthm_info[:,6].flatten()
        self.G_SD_cost =  self.Genthm_info[:,7].flatten()
        
        # Minimum on and off time periods for unit i.
        self.UT = np.int_(self.Genthm_info[:,8].flatten())
        self.DT = np.int_(self.Genthm_info[:,9].flatten())
        
        # Gen inital status code 0(off) / 1(on) 
        self.u_0 =  self.Genthm_info[:,10].flatten()
        
        # The time intervals that the generator has turned ON/OFF(V_{i,0}/S_{i,0}) after 0. (Only for One Bin Model)
        # This param is user-defined, which is not retrieved from
        self.G_Status_on = self.UT
        self.G_Status_off = self.DT
        
        
        
        # operation cost coefficient
        # self.cost_st = self.Genthm_info[:, 11].flatten()
        
        # start up cost coefficient, fix cost for u $Gamma$
        self.cost_fix = self.Genthm_info[:, 12].flatten() 
        
        # start up cost coefficient, fix cost for p $Beta$
        self.costOf = self.Genthm_info[:, 13].flatten() 
        
                                    #1	    2
                                    #bus	pmax	
        self.Windgen_info = np.array([[6,	800],
                                      [5,	800],
                                      [5,	500]])
            
        # Bus Set of renewable energy resources and loads
        self.Wb = self.Genthm_info.shape[0]
        self.Wb_list = self.Windgen_info[:,0].flatten()
        
        # Set of renewable energy resources max power
        self.Wb_p_list = self.Windgen_info[:,1].flatten()
        
        
    def graph_(self, *args):

        G = nx.DiGraph()
        fig = plt.figure(1, figsize=(25, 13), dpi=60)
        
        # add edges
        for index, bus_ft in enumerate(self.Tx_list):
            G.add_edge(int(bus_ft[0]), int(bus_ft[1]))
            G[bus_ft[0]][bus_ft[1]]['weight'] = 5 * self.Tx_cap_list[index]/np.max(self.Tx_cap_list)
        
        # Either define the layout by user or choose the layout provided by nx    
        #pos = nx.spring_layout(G)
        pos = {1: (0, 0), 2: (0, 0.2), 3: (2, 0.2), 4: (-2, 0), 5: (5, 0.03),6: (-6, 0.03)}

        # node, edge, label, node drawing options
        options1 = {
            "width": 4,
        }
        options2 = {
            "edgecolors": "tab:gray", 
            "node_size": 800, 
            "alpha": 1,
            "node_size": 9000,
        }

        options2 = {
            "edgecolors": "tab:gray", 
            "node_size": 800, 
            "alpha": 1,
            "node_size": 9000,
        }
        
        # Some other drawing options
        #nx.draw(G, pos, with_labels=True,connectionstyle='arc3, rad = 0.2',**options1)
        #nx.draw_networkx_edges(G, pos, **options1)
        #nx.draw_networkx(G, pos)
        #nx.draw_networkx(G, **options)
        
        
        widths = list(nx.get_edge_attributes(G,'weight').values())
        nx.draw_networkx_edges(G, pos, width = widths, label="S", arrowstyle='-|>')
        nx.draw_networkx_nodes(G, pos, **options2)
        

        labels = {}
        for i in range(len(pos)):
            if i + 1 in self.Genthm_info[:,:1].flatten():
                labels[i + 1] = f"Thermal G {i + 1}"
            else:
                labels[i + 1] = f"{i + 1}"
        nx.draw_networkx_labels(G, pos, labels, font_size=15, font_color="whitesmoke")

        graph = nx.draw_networkx_edge_labels(G,pos,edge_labels = self.nx_edge_label, font_size=15)

        # Set margins for the axes so that nodes aren't clipped
        ax = plt.gca()
        ax.margins(0.1)
        plt.axis("off")
        plt.show()
        
       
       
        # total numbers of unit
        # Set of regular(r), flexible(f), and all generation resources(I), i.e G = Gr ⋃ Gf .?

        #self.G_f_set = G_list[:f]
        #self.G_r_set = G_list[f:]
        #self.G_B_set = G_list[:1]